# Modelo para determinar o sucesso na oferta de telemarketing de um depósito de longo prazo

Dataset obtido em https://www.kaggle.com/raosuny/success-of-bank-telemarketing-data

**Variáveis de entrada:**

Age (numeric)

Job: career type (categorical: 'admin.', 'blue-collar', 'entrepreneur', 'housemaid', 'management', 'retired', 'self-employed', 'services', 'student', 'technician', 'unemployed')

Marital_Status: marital status (categorical: 'divorced', 'married', 'single'; note: 'divorced' means divorced or widowed)

Education: (categorical: 'PrimaryEducation', 'ProfessionalEducation', 'SecondaryEducation', 'TertiaryEducation')

Default_Credit: has a credit in default? (binary: 'yes', 'no')

Housing_Loan: has a home loan? (binary: 'yes', 'no')

Personal_Loan: has a personal loan? (binary: 'yes', 'no')

**Variável de saída (target):**

Subscribed - has the client subscribed a long-term deposit? (binary: 'yes', 'no')

# **Bibliotecas**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scikitplot as skplt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# **Importando a base**

In [ ]:
df_banco = pd.read_csv('/kaggle/input/success-of-bank-telemarketing-data/Alpha_bank.csv')

# Análise Exploratória

In [ ]:
# Shape do dataframe
df_banco.shape

Temos 8 variáveis e 30477 observações.

In [ ]:
# Pré-visualização dos dados
df_banco.head()

As variáveis binárias, incluindo a variável target, têm valores "yes" e "no". Serão substituídas por 1 e 0.

In [ ]:
# Verificando variáveis - tipos e quantidades
df_banco.info()

Todas as variáveis apresentam 30477 valores, ou seja, não temos missing values.
<br>As variáveis binárias estão definidas como object, pois apresentam valores "yes" e "no". Serão modificadas para o tipo int64.

In [ ]:
# Avaliando outliers
plt.boxplot(df_banco.Age)
plt.title('Boxplot - Idade')
plt.show()

In [ ]:
# Valores nas variáveis
f, ax = plt.subplots(4,2,figsize=(25,35))

plt.subplot(4,2,1)
df_banco.Age.plot(kind='hist', alpha=0.7)
plt.title('Idade')
Age_med = int(df_banco.Age.median())
plt.axvline(x=Age_med, label=f'Mediana:{Age_med}', color='orange')
plt.legend()
plt.subplot(4,2,2)
df_banco.Job.value_counts().plot(kind='barh', alpha=0.7)
plt.title('Trabalho')
plt.subplot(4,2,3)
df_banco.Marital_Status.value_counts().plot(kind='barh', alpha=0.7)
plt.title('Estado Civil')
plt.subplot(4,2,4)
df_banco.Education.value_counts().plot(kind='barh', alpha=0.7)
plt.title('Educação')
plt.subplot(4,2,5)
df_banco.Default_Credit.value_counts().plot(kind='barh', alpha=0.7)
plt.title('Inadimplente')
plt.subplot(4,2,6)
df_banco.Housing_Loan.value_counts().plot(kind='barh', alpha=0.7)
plt.title('Crédito imobiliário')
plt.subplot(4,2,7)
df_banco.Personal_Loan.value_counts().plot(kind='barh', alpha=0.7)
plt.title('Crédito pessoal')
plt.subplot(4,2,8)
df_banco.Subscribed.value_counts().plot(kind='barh', alpha=0.7)
plt.title('Assinou? (Target)')

plt.show()

In [ ]:
# Valores nas variáveis - Clientes que aceitaram oferta
aceito = df_banco[df_banco['Subscribed'] == "yes"]

f, ax = plt.subplots(4,2,figsize=(25,35))

plt.subplot(4,2,1)
aceito.Age.plot(kind='hist', alpha=0.7)
plt.title('Idade')
Age_med = int(aceito.Age.median())
plt.axvline(x=Age_med, label=f'Mediana:{Age_med}', color='orange')
plt.legend()
plt.subplot(4,2,2)
aceito.Job.value_counts().plot(kind='barh', alpha=0.7)
plt.title('Trabalho')
plt.subplot(4,2,3)
aceito.Marital_Status.value_counts().plot(kind='barh', alpha=0.7)
plt.title('Estado Civil')
plt.subplot(4,2,4)
aceito.Education.value_counts().plot(kind='barh', alpha=0.7)
plt.title('Educação')
plt.subplot(4,2,5)
aceito.Default_Credit.value_counts().plot(kind='barh', alpha=0.7)
plt.title('Inadimplente')
plt.subplot(4,2,6)
aceito.Housing_Loan.value_counts().plot(kind='barh', alpha=0.7)
plt.title('Crédito imobiliário')
plt.subplot(4,2,7)
aceito.Personal_Loan.value_counts().plot(kind='barh', alpha=0.7)
plt.title('Crédito pessoal')

plt.show()

Observando os gráficos, verificamos a manutenção das proporções para as variáveis quando selecionamos apenas aqueles que aceitaram a oferta. As exceções são:
* Pessoas com o nível educacional "Tertiary Education" aceitam mais a oferta.
* Trabalhadores "blue collar" e aposentados aceitam mais a oferta.

# Feature Engineering

In [ ]:
# Tratamento dos dados - Replace
#Job: 0 - housemaid, 1 - services, 2 - admin, 3 - technician, 4 - blue-collar, 5 - unemployed, 6 - retired, 7 - entrepreneur,
#     8 - management, 9 - student, 10 - self-employed
df_banco.Job.replace({'housemaid':0, 'services':1, 'admin.':2, 'technician':3, 'blue-collar':4, 'unemployed':5, 'retired':6, 'entrepreneur':7, 'management':8, 'student':9, 'self-employed':10},inplace=True)


#Marital_Status: 0 - married, 1 - single, 2 - divorced
df_banco.Marital_Status.replace({'married':0, 'single':1, 'divorced':2},inplace=True)

#Education: 0 - Primary_Education, 1 - Secondary_Education, 2 - Professional_Education, 3 - Tertiary_Education
df_banco.Education.replace({'Primary_Education':0, 'Secondary_Education':1, 'Professional_Education':2, 'Tertiary_Education':3},inplace=True)


df_banco.Default_Credit.replace({'no':0, 'yes':1},inplace=True)
df_banco.Housing_Loan.replace({'no':0, 'yes':1},inplace=True)
df_banco.Personal_Loan.replace({'no':0, 'yes':1},inplace=True)
df_banco.Subscribed.replace({'no':0, 'yes':1},inplace=True)

In [ ]:
## Faixas de idade

## 0 - Adolescentes: até 20 anos
df_banco.loc[(df_banco['Age'] < 21),'Faixa'] = 0
## 1 - Jovens: 21 aos 30 anos
df_banco.loc[(df_banco['Age'] > 20) & (df_banco['Age'] < 31), 'Faixa'] = 1
## 2 - Meia-idade: 31 aos 59 anos
df_banco.loc[(df_banco['Age'] > 30) & (df_banco['Age'] < 60), 'Faixa'] = 2
## 3 - Idosos: a partir de 60 anos
df_banco.loc[(df_banco['Age'] > 59),'Faixa'] = 3

df_banco['Faixa'] = df_banco['Faixa'].astype(np.int64)

Transformadas todas as variáveis categóricas em numéricas e criada uma variável com faixas de idade.

# Modelo - Treinamento

In [ ]:
# Selecionando variáveis (com idade e não faixas -  e sem variável target)
var = [c for c in df_banco.columns if c not in ['Faixa', 'Subscribed']]

In [ ]:
# Divindo em base de treino, validação e teste
train, test = train_test_split(df_banco, random_state = 42, test_size=0.1)
train, valid = train_test_split(train, random_state = 42, test_size=0.1)

In [ ]:
# Treinando modelo
rf = RandomForestClassifier(n_estimators=200, n_jobs=-1, random_state=42)
rf.fit(train[var], train['Subscribed'])

In [ ]:
# Previsões - Base de validação
preds_val = rf.predict(valid[var])

In [ ]:
# Verificando o desempenho/acurácia - Base de validação
accuracy_score(valid['Subscribed'], preds_val)

In [ ]:
# Previsões - Base de teste
preds_test = rf.predict(test[var])

In [ ]:
# Verificando o desempenho/acurácia - Base de teste
accuracy_score(test['Subscribed'], preds_test)

A princípio o modelo apresenta uma boa acurácia, de 86%.

# Ajustes no modelo

In [ ]:
# Testando percentual de 25% na divisão das bases
train, test = train_test_split(df_banco, random_state = 42, test_size=0.25)
train, valid = train_test_split(train, random_state = 42, test_size=0.25)

# Treinando modelo
rf = RandomForestClassifier(n_estimators=200, n_jobs=-1, random_state=42)
rf.fit(train[var], train['Subscribed'])

In [ ]:
# Previsões - Base de validação
preds_val = rf.predict(valid[var])
# Verificando o desempenho/acurácia - Base de validação
accuracy_score(valid['Subscribed'], preds_val)

In [ ]:
# Previsões - Base de teste
preds_test = rf.predict(test[var])

# Verificando o desempenho/acurácia - Base de teste
accuracy_score(test['Subscribed'], preds_test)

A utilização do percentual de 25% para a divisão das bases de treino, validação e teste piorou o resultado. Portanto retornamos para o percentual de 10%.

In [ ]:
# Testando a utilização de faixas de idade ao invés de idade
# Selecionando variáveis (com idade e não faixas -  e sem variável target)
var = [c for c in df_banco.columns if c not in ['Age', 'Subscribed']]

# Dividindo bases
train, test = train_test_split(df_banco, random_state = 42, test_size=0.1)
train, valid = train_test_split(train, random_state = 42, test_size=0.1)

# Treinando modelo
rf = RandomForestClassifier(n_estimators=200, n_jobs=-1, random_state=42)
rf.fit(train[var], train['Subscribed'])

In [ ]:
# Previsões - Base de validação
preds_val = rf.predict(valid[var])
# Verificando o desempenho/acurácia - Base de validação
accuracy_score(valid['Subscribed'], preds_val)

In [ ]:
# Previsões - Base de teste
preds_test = rf.predict(test[var])

# Verificando o desempenho/acurácia - Base de teste
accuracy_score(test['Subscribed'], preds_test)

A utilização de faixas de idade ao invés da própria idade melhorou o resultado. Portanto mantemos a variável "Faixa".

In [ ]:
# Testando aumentar o número de estimadores
# Treinando modelo
rf = RandomForestClassifier(n_estimators=300, n_jobs=-1, random_state=42)
rf.fit(train[var], train['Subscribed'])

In [ ]:
# Previsões - Base de validação
preds_val = rf.predict(valid[var])
# Verificando o desempenho/acurácia - Base de validação
accuracy_score(valid['Subscribed'], preds_val)

In [ ]:
# Previsões - Base de teste
preds_test = rf.predict(test[var])

# Verificando o desempenho/acurácia - Base de teste
accuracy_score(test['Subscribed'], preds_test)

A utilização de maior número de estimadores não surtiu nenhum efeito no resultado.

In [ ]:
# Testando diminuir o número de estimadores
# Treinando modelo
rf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=42)
rf.fit(train[var], train['Subscribed'])

In [ ]:
# Previsões - Base de validação
preds_val = rf.predict(valid[var])
# Verificando o desempenho/acurácia - Base de validação
accuracy_score(valid['Subscribed'], preds_val)

In [ ]:
# Previsões - Base de teste
preds_test = rf.predict(test[var])

# Verificando o desempenho/acurácia - Base de teste
accuracy_score(test['Subscribed'], preds_test)

A utilização de menor número de estimadores piorou um pouco o resultado. Portanto mantemos o número de 200 estimadores.

# Resultados e predições

Os melhores resultados obtidos são os mostrados abaixo, com a utilização de faixas de idade, divisão das bases de validação e teste no percentual de 10% e número de estimadores para Random Forest de 200.

In [ ]:
# Treinando modelo
rf = RandomForestClassifier(n_estimators=200, n_jobs=-1, random_state=42)
rf.fit(train[var], train['Subscribed'])

In [ ]:
# Previsões - Base de validação
preds_val = rf.predict(valid[var])
# Verificando o desempenho/acurácia - Base de validação
accuracy_score(valid['Subscribed'], preds_val)

In [ ]:
# Previsões - Base de teste
preds_test = rf.predict(test[var])

# Verificando o desempenho/acurácia - Base de teste
accuracy_score(test['Subscribed'], preds_test)

In [ ]:
# Peso de cada variável no resultado
fig = plt.figure(figsize=(15, 5))
pd.Series(rf.feature_importances_, index=var).sort_values().plot.barh()

As variávels Job(trabalho), Faixa(faixa de idade) e Education(nível de ensino) são aquelas que têm maior peso no modelo.

In [ ]:
# Matriz de Confusão - Dados de Teste
skplt.metrics.plot_confusion_matrix(test['Subscribed'], preds_test)

Apesar de uma boa acurácia (87%), a matriz de confusão evidencia que o modelo prevê majoritariamente o valor zero, ou seja, não assinatura. Observamos que das 387 ocorrências de assinatura/contratação, o modelo previu corretamente apenas 13, ou seja temos uma sensibilidade de apenas 3,36%.